# O objetivo da camada gold é:

- Contém dados preparados e otimizados para análise e consumo pelas áreas de negócio. 
- Os dados são agregados e formatados para uso em dashboards, relatórios e outras ferramentas de análise. 
- Esta camada normalmente possui tabelas já prontas para consultas, facilitando a tomada de decisões.
- Apresenta dados analíticos e transacionais consolidados, prontos para serem utilizados pelos usuários finais.
- Região Desconhecida deve ser tratada.
- Criação de tabela consumo de energia com informações de cliente e clima. 

In [1]:
import pandas as pd
import duckdb

import mlflow
from mlflow.tracking import MlflowClient

# Extração

- Tabela Consumo
- Tabela Clima
- Tabela Clientes

In [2]:
# Caminho do banco
db_path = "../../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

# Carrega os dados da camada bronze
clientes_df = con.execute("SELECT * FROM silver.clientes").df()
consumo_df = con.execute("SELECT * FROM silver.consumo").df()
clima_df = con.execute("SELECT * FROM silver.clima").df()

In [3]:
# Cria schema 'silver' se não existir
con.execute("CREATE SCHEMA IF NOT EXISTS gold")

# Transformação

In [4]:
cliente_consumo_df = pd.merge(consumo_df, clientes_df, on='client_id', how='inner')


clientes_df = cliente_consumo_df[cliente_consumo_df['region'] != 'Desconhecida']

In [5]:
consumo_geral = (
    cliente_consumo_df
    .merge(
        clima_df,
        on=['region', 'date'], 
        how='inner'
    )
)

consumo_geral = consumo_geral.sort_values(by=['region', 'date'])


# Carregamento

In [6]:
consumo_geral = consumo_geral[['date', 'client_id', 'region', 'temperature', 'humidity', 'consumption_kwh']]

In [7]:
# Cria tabelas dentro do schema gold
con.execute("""
CREATE TABLE IF NOT EXISTS gold.consumo_geral (
    date DATE,
    client_id VARCHAR,
    region VARCHAR,
    temperature DOUBLE,
    humidity DOUBLE,
    consumption_kwh DOUBLE
)
""")

In [8]:
# Limpa dados se as tabelas já existirem
con.execute("DELETE FROM gold.consumo_geral")

In [9]:
# Registra como tabelas temporárias
con.register("consumo_geral", consumo_geral)

In [10]:
# Insere os dados nas tabelas gold
con.execute("INSERT INTO gold.consumo_geral SELECT * FROM consumo_geral")

In [11]:
con.close()
print("✔ Dados inseridos com sucesso na camada gold do DuckDB.")

✔ Dados inseridos com sucesso na camada gold do DuckDB.
